## Libraries for Modeling

In [25]:
# Libraries
import pandas as pd
import numpy as np
from datetime import datetime, time

## Read CSV File

In [ ]:
csv_file_path = '/Documents/School/Kansas-City-Crime-Analysis/Data/Data_Sets/KCPD-5-Year-Analysis-Feature-Eng.CSV'
df = pd.read_csv(csv_file_path)
print(df.info())

In [27]:
print(df.head(n=5))

    Report_No  Year  Quarter  Month Day_of_Week From_Date From_Time  \
0  KC19020397  2019        1  March   Wednesday   3/20/19  09:00:00   
1  KC19025235  2019        2  April      Sunday    4/7/19  15:45:00   
2  KC19036511  2019        2    May    Thursday   5/16/19  20:30:00   
3  KC19024315  2019        2  April   Wednesday    4/3/19  21:30:00   
4  KC19035992  2019        2    May    Thursday   5/16/19  08:15:00   

  Adjusted_To_Date Adjusted_To_Time General _Offense_Categorization  \
0          3/20/19         13:55:00              Aggravated Assault   
1           4/7/19         15:52:00              Aggravated Assault   
2          5/16/19         21:00:00                         Robbery   
3           4/4/19         01:30:00              Aggravated Assault   
4          5/16/19         08:17:00                  Other Assaults   

       Type_of_Crime UCR_Offense_Classification NIBRS NIBRS Offense Group  \
0      Violent Crime                     Part I   13A             Gro

## Binary Encoding
Binary Encoding presents each category as a binary digit.

**Type_of_Crime** will be handled this way:
**Violent Crime** = 1
**Non-Violent Crime** = 0

In [42]:
# create column Is_Violent with binary encoding for Type_of_Crime column
df['Is_Violent'] = df['Type_of_Crime'].apply(lambda x: 1 if x == 'Violent Crime' else 0)

# define lists of offenses for each category
property_crimes = [
    'Burglary','Larcency-theft','Motor Vehicle Theft', 'Arson', 'Burglary/Breaking and Entering',
    'Vandalism', 'Embezzlement', 'Extortion/Blackmail', 'Fraud','Counterfeiting / Forgery',
    'Robbery', 'Stolen Property Offenses'
    ]
crimes_against_person = ['Murder and Nonnegligent Manslaughter', 'Negligent Manslaughter',
    'Rape', 'Sex Offenses', 'Aggravated Assault', 'Kidnapping/Abduction'
    ]
crimes_against_society = [crime for crime in df['General _Offense_Categorization'].unique()
                          if crime not in property_crimes + crimes_against_person]

# create binary feature for property crime
df['Is_Property_Crime'] = df['General _Offense_Categorization'].apply(lambda x: 1 if x in property_crimes else 0)
df['Is_Crimes_Against_Person'] = df['General _Offense_Categorization'].apply(lambda x: 1 if x in crimes_against_person else 0)
df['Is_Crimes_Against_Society'] = df ['General _Offense_Categorization'].apply(lambda x: 1 if x in crimes_against_society else 0)


## Create Feature - Duration
Represents the length of time in minutes from start to the end time of an event. 

In [28]:
# Convert to datetime
df['From_DateTime'] = pd.to_datetime(df['From_Date'] + ' ' + df['From_Time'])
df['Adjusted_To_DateTime'] = pd.to_datetime(df['Adjusted_To_Date'] + ' ' + df['Adjusted_To_Time'])

# Calculate duration
df['Duration'] = (df['Adjusted_To_DateTime'] - df['From_DateTime']).dt.total_seconds() / 60

# View the DataFrame
print(df[['From_DateTime', 'Adjusted_To_DateTime', 'Duration']])


             From_DateTime Adjusted_To_DateTime  Duration
0      2019-03-20 09:00:00  2019-03-20 13:55:00     295.0
1      2019-04-07 15:45:00  2019-04-07 15:52:00       7.0
2      2019-05-16 20:30:00  2019-05-16 21:00:00      30.0
3      2019-04-03 21:30:00  2019-04-04 01:30:00     240.0
4      2019-05-16 08:15:00  2019-05-16 08:17:00       2.0
...                    ...                  ...       ...
445284 2023-10-08 22:12:00  2023-10-08 22:13:00       1.0
445285 2023-11-01 22:00:00  2023-11-02 19:00:00    1260.0
445286 2023-10-08 23:19:00  2023-10-08 23:19:00       0.0
445287 2023-10-17 14:47:00  2023-10-17 14:46:00      -1.0
445288 2023-10-28 00:00:00  2023-10-30 00:00:00    2880.0

[445289 rows x 3 columns]


## Extract Hour to Capture Time of Day

In [29]:
# Extract hour from From_Time & Adjusted_To_Time
df['From_Time'] = pd.to_datetime(df['From_Time'],format='%H:%M:%S').dt.hour
df['Adjusted_To_Time'] = pd.to_datetime(df['Adjusted_To_Time'], format='%H:%M:%S').dt.hour

print(df.head(n=5))

    Report_No  Year  Quarter  Month Day_of_Week From_Date  From_Time  \
0  KC19020397  2019        1  March   Wednesday   3/20/19          9   
1  KC19025235  2019        2  April      Sunday    4/7/19         15   
2  KC19036511  2019        2    May    Thursday   5/16/19         20   
3  KC19024315  2019        2  April   Wednesday    4/3/19         21   
4  KC19035992  2019        2    May    Thursday   5/16/19          8   

  Adjusted_To_Date  Adjusted_To_Time General _Offense_Categorization  \
0          3/20/19                13              Aggravated Assault   
1           4/7/19                15              Aggravated Assault   
2          5/16/19                21                         Robbery   
3           4/4/19                 1              Aggravated Assault   
4          5/16/19                 8                  Other Assaults   

       Type_of_Crime UCR_Offense_Classification NIBRS NIBRS Offense Group  \
0      Violent Crime                     Part I   13A    

## Cyclical Encoding
Cyclical Encoding of the 'Months' column into sine & cosine values as new columns in the DataFrame.

In [32]:
# Dictionary to map Month string to numerical value
month_to_num = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7,'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Map the 'Month' column to numeric value
df['Month'] = df['Month'].map(month_to_num)

# Dictionary to map Day string to numerical value
day_to_num = {
    'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3,
    'Thursday': 4, 'Friday': 5, 'Saturday': 6
}

# Map the 'Day_of_Week' column to numeric value
df['Day_of_Week'] = df['Day_of_Week'].map(day_to_num)

# Apply cyclical encoding to Year
df['sin_Quarter'] = np.sin((2 * np.pi * df['Quarter'])/4)
df['cos_Quarter'] = np.sin((2 * np.pi * df['Quarter'])/4)

# Apply cyclical encoding to Months
df['sin_Months'] = np.sin(2 * np.pi * df['Month']/12.0)
df['cos_Months'] = np.cos(2 * np.pi * df['Month']/12.0)

# Apply cyclical encoding to Day of Week for Weekday vs. Weekend Analysis
df['sin_DayOfWeek'] = np.sin(2 * np.pi * df['Day_of_Week']/7.0)
df['cos_DayOfWeek'] = np.cos(2 * np.pi * df['Day_of_Week']/7.0)

# Apply cyclical encoding to From_Time
df['sin_From_Time'] = np.sin(2 * np.pi * df['From_Time']/24)
df['cos_From_Time'] = np.cos(2 * np.pi * df['From_Time']/24)

# Apply cyclical encoding to Adjusted_To_Time
df['sin_Adj_To_Time'] = np.sin(2 * np.pi * df['Adjusted_To_Time']/24)
df['cos_Adj_To_Time'] = np.cos(2 * np.pi * df['Adjusted_To_Time']/24)


print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445289 entries, 0 to 445288
Data columns (total 26 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Report_No                        445289 non-null  object        
 1   Year                             445289 non-null  int64         
 2   Quarter                          445289 non-null  int64         
 3   Month                            445289 non-null  int64         
 4   Day_of_Week                      445289 non-null  int64         
 5   From_Date                        445289 non-null  object        
 6   From_Time                        445289 non-null  int64         
 7   Adjusted_To_Date                 445289 non-null  object        
 8   Adjusted_To_Time                 445289 non-null  int64         
 9   General _Offense_Categorization  445289 non-null  object        
 10  Type_of_Crime                    445289 non-

## One-Hot Encoding

In [45]:
# Adding year trend
df['Year_Quarter'] = df['Year'].astype(str) + '_Q' + df['Quarter'].astype(str)

year_quarter_dummies = pd.get_dummies(df['Year_Quarter'])
df = pd.concat([df, year_quarter_dummies], axis=1)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445289 entries, 0 to 445288
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Report_No                        445289 non-null  object        
 1   Year                             445289 non-null  int64         
 2   Quarter                          445289 non-null  int64         
 3   Month                            445289 non-null  int64         
 4   Day_of_Week                      445289 non-null  int64         
 5   From_Date                        445289 non-null  object        
 6   From_Time                        445289 non-null  int64         
 7   Adjusted_To_Date                 445289 non-null  object        
 8   Adjusted_To_Time                 445289 non-null  int64         
 9   General _Offense_Categorization  445289 non-null  object        
 10  Type_of_Crime                    445289 non-

## Ordinal Encoding
This is meant to create a mapping for **General_Offense_Classification** that assigns a rank order to each type of offence based on the Uniform Crime Reporting program classification of Part I and Part 2 offenses. 

In [39]:
# offense ranking according to the UCR hierarchy
offense_ranking = {
    'Murder and Nonnegligent Manslaughter': 1,
    'Negligent Manslaughter': 2,
    'Rape': 3,
    'Robbery': 4,
    'Aggravated Assault': 5,
    'Burglary/Breaking and Entering': 6,
    'Larceny-theft': 7,
    'Motor Vehicle Theft': 8,
    'Arson': 9,
    'Other Assaults': 10,
    'Counterfeiting / Forgery': 11,
    'Fraud': 12,
    'Embezzlement': 13,
    'Extortion/Blackmail': 14,
    'Stolen Property Offenses': 15,
    'Vandalism': 16,
    'Weapons: Carrying, Possessing, etc.': 17,
    'Prostitution and Commercialized Vice': 18,
    'Pornography / Obscene Material': 19,
    'Sex Offenses': 20,
    'Drug Abuse Violations': 21,
    'Drug Equipment Violations': 22,
    'Offenses Against Family and Children': 23,
    'Driving Under the Influence': 24,
    'Liqour Laws': 25,
    'Drunkenness': 26,
    'Disorderly Conduct': 27,
    'All Other Offenses': 28,
    'Kidnapping/Abduction': 29,
    'Bribery': 30,
    'Curfew and Loitering Laws': 31,
    'Animal Cruelty': 32
}

# Assign the rank to each offense
df['Offense_Rank'] = df['General _Offense_Categorization'].map(offense_ranking)

In [46]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445289 entries, 0 to 445288
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Report_No                        445289 non-null  object        
 1   Year                             445289 non-null  int64         
 2   Quarter                          445289 non-null  int64         
 3   Month                            445289 non-null  int64         
 4   Day_of_Week                      445289 non-null  int64         
 5   From_Date                        445289 non-null  object        
 6   From_Time                        445289 non-null  int64         
 7   Adjusted_To_Date                 445289 non-null  object        
 8   Adjusted_To_Time                 445289 non-null  int64         
 9   General _Offense_Categorization  445289 non-null  object        
 10  Type_of_Crime                    445289 non-

## Drop Columns

In [47]:
# columns to drop
columns_to_drop = [
    'Report_No', 'Year', 'Quarter', 'Month', 'Day_of_Week',
    'From_Date', 'Adjusted_To_Date', 'From_Time', 'Adjusted_To_Time',
    'General _Offense_Categorization', 'Type_of_Crime',
    'UCR_Offense_Classification', 'NIBRS', 'NIBRS Offense Group','From_DateTime',
    'Adjusted_To_DateTime', 'Year_Quarter'
]

# drop specified columns
df.drop(columns=columns_to_drop, inplace=True)

In [49]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445289 entries, 0 to 445288
Data columns (total 35 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Age_Range                  445289 non-null  object 
 1   Duration                   445289 non-null  float64
 2   sin_Months                 445289 non-null  float64
 3   cos_Months                 445289 non-null  float64
 4   sin_DayOfWeek              445289 non-null  float64
 5   cos_DayOfWeek              445289 non-null  float64
 6   sin_From_Time              445289 non-null  float64
 7   cos_From_Time              445289 non-null  float64
 8   sin_Adj_To_Time            445289 non-null  float64
 9   cos_Adj_To_Time            445289 non-null  float64
 10  Offense_Rank               445289 non-null  int64  
 11  Is_Violent                 445289 non-null  int64  
 12  Is_Property_Crime          445289 non-null  int64  
 13  Is_Crimes_Against_Person   44

## Export DataFrame to New CSV

In [55]:
# file path for the new file
file_path = 'KCPD-5-Year-Analysis-Model.csv'

# export the dataframe
df.to_csv(file_path,index=False)